# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *O*

**Names:**

* *Argelaguet Franquelo, Pau*
* *du Bois de Dunilac, Vivien*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [53]:
import pickle
import numpy as np
import string
import collections
import operator

from functools import reduce
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

from nltk.stem import SnowballStemmer

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Exercise 4.1: Pre-processing

In [8]:
stemmer = SnowballStemmer("english")

In [23]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def filter_word(word):
    # Removing words in stopwords
    if word in stopwords:
        return False
    # Removing words consisting of a punctuation sign
    if word in string.punctuation:
        return False
    # Removing numbers
    if is_number(word):
        return False
    return True


def clean_word(word):
    # Removing punctuation signs from word
    word = "".join(c for c in word if c not in string.punctuation)
    # Transforming word to lowercase
    word = word.lower()
    # Stemming
    word = stemmer.stem(word)
    return word
    

def get_bag_of_words(text):
    words = filter(filter_word, map(clean_word, text.split()))
    bow = {}
    for w in words:
        if w in bow:
            bow[w] += 1
        else:
            bow[w] = 1
    
    # Removing less frequent words
    bow = {k: v for k, v in bow.items() if v > 1}
            
    return dict(collections.OrderedDict(sorted(bow.items())))

In [24]:
dat = {
    x.get('courseId'): {
        'name': x.get('name'),
        'description': get_bag_of_words(x.get('description'))
    } for x in courses
}

In [95]:
list_terms = list(x.get('description') for x in dat.values())

terms = {}
for l in list_terms:
    for k, v in l.items():
        if k in terms:
            terms[k] += v
        else:
            terms[k] = v
            
terms = dict(collections.OrderedDict(sorted(terms.items(), key=operator.itemgetter(1), reverse=True)))
top_terms = list({k: v for k, v in terms.items() if v > 400}.keys())
low_terms = list({k: v for k, v in terms.items() if v < 3}.keys())

In [96]:
for k, v in dat.items():
    dat[k]['description'] = {x: y for x, y in v.get('description').items() if x not in top_terms}

In [97]:
dat['COM-308']['description']

{'ad': 2,
 'algebra': 2,
 'algorithm': 2,
 'analyt': 2,
 'applic': 2,
 'auction': 2,
 'base': 2,
 'class': 3,
 'cluster': 2,
 'communiti': 2,
 'comput': 2,
 'data': 6,
 'dataset': 2,
 'detect': 2,
 'ecommerc': 2,
 'explor': 5,
 'graph': 2,
 'hadoop': 2,
 'homework': 2,
 'inform': 2,
 'internet': 2,
 'lab': 3,
 'largescal': 3,
 'linear': 2,
 'machin': 2,
 'mine': 3,
 'network': 4,
 'number': 2,
 'onlin': 5,
 'practic': 2,
 'problem': 2,
 'realworld': 4,
 'recommend': 3,
 'relat': 2,
 'servic': 3,
 'session': 2,
 'social': 5,
 'stream': 2}

In [100]:
with open("data/preprocess.pckl", "wb") as f:
    pickle.dump(dat, f)

## Exercise 4.2: Term-document matrix

In [93]:
list_terms = [list(x.get('description').keys()) for x in dat.values()]

terms = set()
for l in list_terms:
    terms = terms.union(set(l))

In [38]:
documents = list(dat.keys())

## Exercise 4.3: Document similarity search